In [11]:
import sys
project_root = "/home/bobby/repos/latent-neural-dynamics-modeling"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Exploratory Data Analysis To-Do List

This checklist outlines the key steps for the initial exploratory data analysis (EDA) based on the project proposal.


### 1. Setup and Initial Data Loading

* Import necessary libraries (`numpy`,  `scipy`,  `pandas`,  `matplotlib`,  `mne`, etc.).
* Define file paths and constants.
* Load data for a single participant and session to establish the workflow.


In [10]:
from pathlib import Path
import plotly.express as px
import polars as pl

from utils.file_handling import get_child_subchilds_tuples

In [14]:
ROOT_PATH = Path(project_root)
DATA_PATH = ROOT_PATH / "resampled_recordings"
PARTICIPANTS_PATH = DATA_PATH / "participants"

In [37]:
def load_participant_session_data(participant_id: str, session_id: str) -> pl.DataFrame:
    p_partition_path = PARTICIPANTS_PATH / participant_id / session_id / "*"
    print(f"Loading data from: {p_partition_path}")
    return pl.read_parquet(p_partition_path)

In [31]:
participants_partitions = get_child_subchilds_tuples(PARTICIPANTS_PATH)
participants_partitions

[('participant_id=PDI4', 'session=3'),
 ('participant_id=PDI4', 'session=2'),
 ('participant_id=PDI1', 'session=4'),
 ('participant_id=PDI1', 'session=2'),
 ('participant_id=PDI3', 'session=4'),
 ('participant_id=PDI3', 'session=2'),
 ('participant_id=PDI2', 'session=4'),
 ('participant_id=PDI2', 'session=2')]

In [36]:
participant_id = "participant_id=PDI1"
session_id = "session=2"

In [38]:
participants = load_participant_session_data(participant_id, session_id)
participants

Loading data from: /home/bobby/repos/latent-neural-dynamics-modeling/resampled_recordings/participants/participant_id=PDI1/session=2/*


participant_id,session,block,trial,onset,duration,time,start_ts,trial_length_ts,chunk_margin,dbs_stim,yscore,LFP_1,LFP_2,LFP_3,LFP_4,LFP_5,LFP_6,LFP_7,LFP_8,LFP_9,LFP_10,LFP_11,LFP_12,LFP_13,LFP_14,LFP_15,LFP_16,ECOG_1,ECOG_2,ECOG_3,ECOG_4,x,y,tracing_coordinates_present
str,u32,u32,u32,f64,f32,list[f64],u32,u32,i32,i32,f32,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[i64],list[i64],bool
"""PDI1""",2,1,1,16.102864,11.019526,"[16.102864, 16.103864, … 27.120864]",16102,11019,2,0,2.703554,"[0.00001, 0.000007, … -0.000001]","[0.000009, 0.000005, … -0.000001]","[0.000027, 0.000002, … -0.00003]","[0.000006, 0.000079, … 0.000089]","[0.000004, 3.8836e-7, … -0.000002]","[-0.000028, -0.000034, … -0.000048]","[-0.000022, -0.000053, … -0.000019]","[0.000005, 0.000002, … 0.000002]","[0.000006, 2.1811e-7, … -0.000001]","[5.0463e-7, -0.000005, … 0.000002]","[0.000002, -0.000002, … -5.1700e-7]","[0.000002, -0.000002, … -2.2884e-7]","[0.000005, -7.1312e-7, … 6.5943e-7]","[0.000004, -0.000002, … 0.000003]","[0.000004, -0.000002, … -0.000003]","[0.000004, -0.000001, … -4.0172e-7]","[-0.000019, -0.000023, … 0.000012]","[-0.000017, -0.000018, … 0.000013]","[-0.000015, -0.000015, … 0.00001]","[-0.000013, -0.00001, … 0.000007]","[-15, -113, … -360]","[217, 45, … 47]",true
"""PDI1""",2,1,2,34.896455,11.019526,"[34.896455, 34.897455, … 45.914455]",34896,11019,2,0,1.811423,"[0.000001, -6.8187e-7, … 6.2975e-7]","[-0.000005, -0.000007, … -0.000005]","[0.000009, -0.000004, … -0.000032]","[5.1902e-7, -0.00005, … 0.000014]","[-0.000002, -0.000005, … -0.000006]","[0.00001, 0.00001, … -0.000009]","[-0.000022, -0.000035, … -0.000036]","[-0.000005, -0.000007, … -8.7612e-8]","[-4.5056e-7, -0.000002, … -0.000009]","[0.000001, -0.000001, … -0.000007]","[-0.000003, -0.000004, … -0.000004]","[-0.000005, -0.000007, … -0.000003]","[-0.000002, -0.000005, … -0.000003]","[-0.000004, -0.000006, … -0.000002]","[-0.000012, -0.000014, … 0.000004]","[-0.000006, -0.000008, … 6.7189e-9]","[0.000008, 0.000004, … -0.000005]","[0.000002, -0.000002, … -0.000007]","[0.000004, 2.4216e-7, … -0.00001]","[0.000008, 0.000007, … -0.000009]","[-6, -23, … -301]","[48, -6, … 24]",true
"""PDI1""",2,1,3,53.729091,11.019526,"[53.729091, 53.730091, … 64.747091]",53729,11019,2,0,2.518298,"[0.000004, 0.000006, … -0.000015]","[0.000004, 0.000005, … -0.000027]","[0.00005, 0.000018, … -0.000042]","[-0.000046, -0.00002, … -0.000041]","[0.000004, 0.000006, … -0.000023]","[0.000036, 0.000017, … -0.000023]","[0.000032, 0.000026, … -0.000038]","[0.000005, 0.000006, … -0.000026]","[0.000012, 0.000013, … -0.000019]","[0.000013, 0.000013, … -0.000016]","[0.000014, 0.000015, … -0.000023]","[0.00001, 0.000011, … -0.000026]","[0.000016, 0.000017, … -0.000028]","[-0.000005, -0.000004, … -0.000032]","[-0.000005, -0.000004, … -0.000043]","[0.000008, 0.000009, … -0.000029]","[0.000014, 0.000012, … 0.000009]","[0.000021, 0.000021, … 0.000016]","[0.000017, 0.000017, … 0.000014]","[0.000012, 0.000012, … 0.00001]","[-281, -209, … 3]","[-1, 110, … -193]",true
"""PDI1""",2,1,4,70.946727,11.019526,"[70.946727, 70.947727, … 81.964727]",70946,11019,2,0,2.052713,"[-0.000001, 0.000003, … 0.00002]","[-0.000005, -9.3995e-7, … 0.000018]","[-0.000016, -0.000022, … -0.000024]","[-0.000067, -0.000082, … -0.000057]","[-0.000002, 0.000001, … 0.000021]","[-0.000024, -0.000007, … 0.000027]","[0.000002, 0.000026, … 0.00002]","[-0.000003, 1.3734e-7, … 0.000019]","[-0.000008, -0.000004, … 0.000006]","[-0.000006, -0.000002, … 0.000003]","[4.5071e-7, 0.000004, … 0.000009]","[-0.000004, -6.8121e-7, … 0.000006]","[-0.000004, -0.000001, … 0.000007]","[-0.000005, -0.000003, … 0.00001]","[-0.000016, -0.000013, … 0.000006]","[-0.000004, -0.000001, … 0.00001]","[-0.000003, -0.000013, … -0.000015]","[-0.000007, -0.000018, … -0.000007]","[-0.000006, -0.000015, … -0.000002]","[-0.000013, -0.00

### 2. Signal Quality Check

* **Visual Inspection of Raw Time-Series:**
    - Plot a segment of raw LFP data.
    - Plot a segment of raw ECoG data.
    - Check for any obvious issues like flat channels, clipping, or high-amplitude noise.
* **Visualize Recordings with DBS Markers:**
    - Load the event markers for DBS ON/OFF conditions.
    - Create plots of the LFP and ECoG signals, overlaying markers or using shaded regions to indicate the different DBS states.
* **Verify Artifact Removal:**
    - Load a segment of data from a DBS ON block.
    - Plot the signal before applying the template subtraction method.
    - Plot the signal *after* applying template subtraction to visually confirm that the stimulation artifact is removed or significantly reduced.


### 3. Power Spectral Density (PSD) Analysis (for RQ2)

* **Segment Data:** Separate the preprocessed data into epochs corresponding to DBS ON and DBS OFF conditions.
* **Calculate PSD:**
    - Compute PSD estimates for LFP channels during DBS ON vs. OFF.
    - Compute PSD estimates for ECoG channels during DBS ON vs. OFF.
* **Compare PSDs:**
    - Plot the mean PSD for ON vs. OFF conditions for both LFP and ECoG to identify potential neural markers (e.g., changes in beta-band power).
* **Statistical Testing:**
    - Implement or use a library function for the cluster-based permutation test.
    - Run the test on the PSD data to find significant differences between DBS ON and OFF states.


### 4. Behavioral Data Processing (for RQ3)

* **Load Kinematic Data:** Load the hand position coordinate data for a trial.
* **Calculate Instantaneous Speed:**
    - Take the first derivative of the position data (e.g., using `np.diff`).
    - Calculate the Euclidean norm of the velocity vectors to get instantaneous speed.
* **Smooth the Speed Signal:**
    - Apply a moving average filter to the instantaneous speed signal.
* **Visualize Behavioral Trace:**
    - Plot the final, smoothed tracing speed signal to verify it looks like a reasonable continuous behavioral trace.
